__Notes__: This notebook is deprecated.

# Classification of comments as ad-hominem and delta using BERT
Before running this notebook, make sure you have already finished running `1-prepare-dataset.ipynb` 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
!pip install -q tensorflow-text
!pip install -q tf-models-official

     |████████████████████████████████| 3.4MB 18.7MB/s 
     |████████████████████████████████| 1.1MB 16.1MB/s 
     |████████████████████████████████| 37.6MB 80kB/s 
     |████████████████████████████████| 358kB 57.1MB/s 
     |████████████████████████████████| 645kB 53.7MB/s 
     |████████████████████████████████| 706kB 54.2MB/s 
     |████████████████████████████████| 1.2MB 40.8MB/s 
     |████████████████████████████████| 102kB 12.1MB/s 
     |████████████████████████████████| 174kB 55.5MB/s 
     |████████████████████████████████| 51kB 8.5MB/s 


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optmizer

tf.get_logger().setLevel('ERROR')

In [ ]:
import os

if tf.test.is_gpu_available():
  strategy = tf.distribute.MirroredStrategy()
  print('Using GPU')
else:
  raise ValueError('Running on CPU is not recomended.')

Using GPU


In [ ]:
!nvidia-smi

Fri Apr  9 21:36:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    28W /  70W |    222MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
os.environ["TFHUB_MODEL_LOAD_FORMAT"]="UNCOMPRESSED"

In [ ]:
train_dir = '/content/gdrive/MyDrive/DL/dataset/bert/train/'
test_dir = '/content/gdrive/MyDrive/DL/dataset/bert/test/'

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
seed = 42

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.7,
    subset='training',
    seed=seed)

class_names = raw_train_ds.class_names
train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)

test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    test_dir,
    batch_size=batch_size)

test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

Found 5242 files belonging to 2 classes.
Using 1573 files for training.
Found 2000 files belonging to 2 classes.


In [ ]:
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3'
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [ ]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [ ]:
history = None

with strategy.scope():
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    metrics = tf.metrics.BinaryAccuracy()
    epochs = 5
    steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
    num_train_steps = steps_per_epoch * epochs
    num_warmup_steps = int(0.1*num_train_steps)

    init_lr = 3e-5
    optimizer = optimization.create_optimizer(init_lr=init_lr, num_train_steps=num_train_steps, num_warmup_steps=num_warmup_steps, optimizer_type='adamw')
    classifier_model = build_classifier_model()
    classifier_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    print(f'Training model with {tfhub_handle_encoder}')
    history = classifier_model.fit(x=train_ds, epochs=epochs)

Training model with https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
Epoch 1/5
50/50 [==============================] - 135s 2s/step - loss: 0.6904 - binary_accuracy: 0.5320
Epoch 2/5
50/50 [==============================] - 46s 911ms/step - loss: 0.4553 - binary_accuracy: 0.7940
Epoch 3/5
50/50 [==============================] - 45s 891ms/step - loss: 0.3437 - binary_accuracy: 0.8596
Epoch 4/5
50/50 [==============================] - 45s 899ms/step - loss: 0.1865 - binary_accuracy: 0.9305
Epoch 5/5
50/50 [==============================] - 45s 895ms/step - loss: 0.1310 - binary_accuracy: 0.9583


In [ ]:
class Comment:
    '''Container storing comments and their labels as predicted by various models'''
    def __init__(self, cid, text, label):
        self.id = cid
        self.text = text
        self.label = label
        self.labels = dict()
        self.scores = dict()
        self.ah = dict()

    def __str__(self):
        s = 'id: ' + str(self.id) + '\n'
        s += 'comment: ' + self.text + '\n'
        s += 'label: ' + self.label + '\n'
        return s

    def add_model(self, model, label, score, words):
        self.labels[model] = label
        self.scores[model] = score
        self.ah[model] = words

In [ ]:
import numpy as np

# define softmax function
def softmax(x):
    return 1 / (1 + np.exp(-x))

# define prediction function
def predict_probs(texts):
    predictions = classifier_model.predict(texts)
    pr = list(predictions)
    nr = list()
    for x in pr:
        y = list(x)
        y = y[0]
        y = softmax(y)
        z = 1 - y
        ff = list()
        ff.append(z)
        ff.append(y)
        ff = np.array(ff)
        nr.append(ff)
    nr = np.array(nr)
    return nr

In [ ]:
!pip install lime

     |████████████████████████████████| 276kB 18.8MB/s 
  Created wheel for lime: filename=lime-0.2.0.1-cp37-none-any.whl size=283846 sha256=c1eb837fd650629a2a56be7683bf17c0db5f4bec5d033563d09cbaae2442abac
  Stored in directory: /root/.cache/pip/wheels/4c/4f/a5/0bc765457bd41378bf3ce8d17d7495369d6e7ca3b712c60c89
Successfully built lime


In [ ]:
print(class_names)

['ah', 'none']


In [ ]:
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names = class_names)

import pickle

In [ ]:
from tqdm import tqdm
from comment import Comment

indir = '/content/gdrive/MyDrive/DL/dataset/comments_6.log'
outdir = '/content/gdrive/MyDrive/DL/dataset/comments_7.log'

reader = open(indir, 'rb')

comments = []

e = Comment(0, 'lol geez', 'ah')

try:
    while True:
        e = pickle.load(reader)
        print(e)
        comments.append(e)
except EOFError:
    reader.close()

Streaming output truncated to the last 5000 lines.
id: 750
comment: love how op just ignored this lmao
label: ah

id: 751
comment: wow i really like this well done
label: none

id: 752
comment: youve already admitted that you are full of shit and completely disinterested in actually learning anything about linguistics im out
label: ah

id: 753
comment: au contraire i dont think ive ever seen a set of people with less open minds funny how obvious that sounds when you say it out loud
label: none

id: 754
comment: you hurt my head i understand that its a single stall bathroom with a lock that does not change the social stigma of a man entering the womens washroom viceversa at all how are you supposed to know its a singlestall from the outside at a quick glance youre not im not questioning or debating the ethics of using a singlestalled gendered bathroom i am not debating anything actually im telling you that when someone sees a guy enter a womens bathroom chances are they might say someth

In [ ]:
for i in tqdm(range(len(comments))):
    text = list()
    text.append(comments[i].text)
    result = tf.sigmoid(classifier_model(tf.constant(text)))
    score = float(result[0][0])
    label = 'none'
    if score < 0.5:
        score = 1 - score
        label = 'ah'
    words = None
    comments[i].add_model('bert-30', label, score, words)

100%|██████████| 2000/2000 [00:28<00:00, 69.25it/s]


In [ ]:
writer = open(outdir, 'wb')
for i in tqdm(range(len(comments))):
    pickle.dump(comments[i], writer)
writer.close()

100%|██████████| 2000/2000 [00:00<00:00, 50048.97it/s]


In [ ]:
for x in comments:
    print(x.labels)
    print(x.scores)
    print("-" * 30)

Streaming output truncated to the last 5000 lines.
{'bert': 0.9378126189112663, 'cnn': 0.67654717, 'bilstm': 0.60124993, 'bert-5': 0.9063384160399437, 'bert-10': 0.8743482530117035, 'bert-20': 0.9896246930584311, 'bert-30': 0.9669492207467556}
------------------------------
{'bert': 'ah', 'cnn': 'ah', 'bilstm': 'none', 'bert-5': 'none', 'bert-10': 'none', 'bert-20': 'none', 'bert-30': 'none'}
{'bert': 0.9754031617194414, 'cnn': 0.6688594, 'bilstm': 0.53095317, 'bert-5': 0.7169830799102783, 'bert-10': 0.5255247950553894, 'bert-20': 0.7439258098602295, 'bert-30': 0.8413074016571045}
------------------------------
{'bert': 'none', 'cnn': 'none', 'bilstm': 'none', 'bert-5': 'none', 'bert-10': 'none', 'bert-20': 'none', 'bert-30': 'none'}
{'bert': 0.9967342615127563, 'cnn': 0.9270566, 'bilstm': 0.9935364, 'bert-5': 0.8011320233345032, 'bert-10': 0.9666491150856018, 'bert-20': 0.9978426694869995, 'bert-30': 0.9975396394729614}
------------------------------
{'bert': 'ah', 'cnn': 'ah', 'bilst